In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 41.2 MB/s eta 0:00:00


In [3]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
#@title Color with slightly intense red and green
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdFMCS, Draw
import os
from io import BytesIO
import base64

# ========== UI SETUP ==========
upload_btn = widgets.FileUpload(description="Upload Excel", accept='.xlsx,.xls', multiple=False)
output_dir_input = widgets.Text(value='preview_output', description='Output folder:')
process_btn = widgets.Button(description="Generate Preview Images", button_style='success')
preview_btn = widgets.Button(description="Preview Structures", button_style='info')
output = widgets.Output()

display(widgets.VBox([
    widgets.HTML("<h2>SMILES to Colored Structure Preview</h2>"),
    widgets.HTML("Upload Excel with columns: <b>Smiles, R1_Reagent, R2_Reagent, R3_Reagent, R4_Reagent</b>"),
    upload_btn,
    output_dir_input,
    widgets.HBox([preview_btn, process_btn]),
    output
]))

# ========== PROCESSING FUNCTIONS ==========
def process_rgroups(row, mol):
    """Identify R-group mappings with proper visualization"""
    atom_colors = {}
    rgroup_info = []

    # Define R-groups with their colors - using lighter blue for R3
    rgroups = [
        ('R1', 'R1_Reagent', (1.0, 0.0, 0.0)),    # Red
        ('R2', 'R2_Reagent', (0.0, 1.0, 0.0)),    # Green
        ('R3', 'R3_Reagent', (0.4, 0.6, 1.0)),    # Lighter Blue (was 0.0, 0.0, 1.0)
        ('R4', 'R4_Reagent', (1.0, 0.0, 1.0))     # Magenta
    ]

    for rname, rcol, color in rgroups:
        if rcol in row and pd.notna(row[rcol]):
            reagent_smiles = str(row[rcol])
            reagent_mol = Chem.MolFromSmiles(reagent_smiles)
            if reagent_mol:
                mcs = rdFMCS.FindMCS([mol, reagent_mol])
                if mcs.numAtoms > 0:
                    substruct = Chem.MolFromSmarts(mcs.smartsString)
                    matches = mol.GetSubstructMatches(substruct)
                    for match in matches:
                        for atom_idx in match:
                            if atom_idx not in atom_colors:
                                atom_colors[atom_idx] = color
                    rgroup_info.append(f"{rname}: {len(matches)} matches")
                else:
                    rgroup_info.append(f"{rname}: No common substructure")
            else:
                rgroup_info.append(f"{rname}: Invalid reagent SMILES")
        else:
            rgroup_info.append(f"{rname}: Not provided")

    return atom_colors, rgroup_info

def orient_molecule(mol, atom_colors):
    """Orient molecule so R1 (red) is on left and R3 (blue) is on right"""
    # First generate 2D coordinates
    AllChem.Compute2DCoords(mol)

    # Get atom positions
    conf = mol.GetConformer()

    # Find centroids of R1 (red) and R3 (blue) atoms
    red_atoms = [idx for idx, color in atom_colors.items() if color == (1.0, 0.0, 0.0)]
    blue_atoms = [idx for idx, color in atom_colors.items() if color == (0.4, 0.6, 1.0)]  # Updated blue color

    if red_atoms and blue_atoms:
        # Calculate centroids
        red_centroid_x = sum(conf.GetAtomPosition(idx).x for idx in red_atoms) / len(red_atoms)
        blue_centroid_x = sum(conf.GetAtomPosition(idx).x for idx in blue_atoms) / len(blue_atoms)

        # If R1 (red) is not on the left side, rotate the molecule
        if red_centroid_x > blue_centroid_x:
            # Rotate 180 degrees around the center
            center_x = sum(conf.GetAtomPosition(i).x for i in range(mol.GetNumAtoms())) / mol.GetNumAtoms()
            center_y = sum(conf.GetAtomPosition(i).y for i in range(mol.GetNumAtoms())) / mol.GetNumAtoms()

            for i in range(mol.GetNumAtoms()):
                pos = conf.GetAtomPosition(i)
                new_x = 2 * center_x - pos.x  # Mirror across center
                new_y = pos.y
                conf.SetAtomPosition(i, (new_x, new_y, 0))

    return mol

def create_custom_template():
    """Create a simple template with R1 on left and R3 on right"""
    # Create a simple scaffold that can serve as a template
    # This is a generic scaffold that encourages left-right orientation
    template_smiles = "C(C)(C)C"  # Simple branched molecule that encourages left-right orientation
    template_mol = Chem.MolFromSmiles(template_smiles)
    if template_mol:
        AllChem.Compute2DCoords(template_mol)
    return template_mol

def align_to_desired_orientation(mol, atom_colors):
    """Align molecule to have R1 on left and R3 on right"""
    try:
        # First try to use a custom template
        template_mol = create_custom_template()
        if template_mol:
            AllChem.GenerateDepictionMatching2DStructure(mol, template_mol)

        # Then apply our custom orientation
        mol = orient_molecule(mol, atom_colors)
        return True
    except:
        # Fallback: just use custom orientation
        try:
            mol = orient_molecule(mol, atom_colors)
            return True
        except:
            AllChem.Compute2DCoords(mol)
            return False

def preview_structure(mol, atom_colors):
    """Generate preview with custom R-group coloring (no shadows)"""
    from rdkit.Chem.Draw import rdMolDraw2D

    # Create drawer
    drawer = rdMolDraw2D.MolDraw2DCairo(400, 400)
    draw_options = drawer.drawOptions()

    # CRITICAL: Disable all highlight effects to remove shadows
    draw_options.fillHighlights = False
    draw_options.highlightRadius = 0

    # Prepare highlight information
    highlight_atoms = list(atom_colors.keys())
    highlight_colors = {}
    for atom_idx, color in atom_colors.items():
        highlight_colors[atom_idx] = color

    # Also color bonds between atoms of the same R-group
    highlight_bonds = []
    bond_colors = {}
    for bond in mol.GetBonds():
        begin_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        if (begin_idx in atom_colors and end_idx in atom_colors and
            atom_colors[begin_idx] == atom_colors[end_idx]):
            highlight_bonds.append(bond.GetIdx())
            bond_colors[bond.GetIdx()] = atom_colors[begin_idx]

    # Draw the molecule
    drawer.DrawMolecule(mol,
                       highlightAtoms=highlight_atoms,
                       highlightAtomColors=highlight_colors,
                       highlightBonds=highlight_bonds,
                       highlightBondColors=bond_colors)

    drawer.FinishDrawing()

    # Convert to base64 for display
    img_data = drawer.GetDrawingText()
    img_str = base64.b64encode(img_data).decode()
    return f'<img src="data:image/png;base64,{img_str}" />'

def create_legend(atom_colors):
    """Create a proper legend showing which colors correspond to which R-groups"""
    color_mapping = {
        (1.0, 0.0, 0.0): "R1 (Red) ← LEFT",
        (0.0, 1.0, 0.0): "R2 (Green)",
        (0.4, 0.6, 1.0): "R3 (Light Blue) → RIGHT",  # Updated description
        (1.0, 0.0, 1.0): "R4 (Magenta)"
    }

    used_colors = set(atom_colors.values())
    legend_html = "<div style='margin: 10px 0; padding: 10px; border: 1px solid #ccc; background: #f9f9f9;'>"
    legend_html += "<strong>R-Group Colors (Oriented: R1←LEFT, R3→RIGHT):</strong><br>"

    for color in used_colors:
        color_name = color_mapping.get(tuple(color), "Unknown R-group")
        hex_color = "#{:02x}{:02x}{:02x}".format(
            int(color[0] * 255),
            int(color[1] * 255),
            int(color[2] * 255)
        )
        legend_html += f"<span style='color: {hex_color}; font-weight: bold; margin-right: 15px;'>■ {color_name}</span>"

    legend_html += "</div>"
    return legend_html

# ========== EVENT HANDLERS ==========
def on_preview_click(b):
    with output:
        clear_output()
        if not upload_btn.value:
            print("⚠️ Please upload an Excel file first")
            return

        try:
            file_info = next(iter(upload_btn.value.values()))
            df = pd.read_excel(BytesIO(file_info['content']))
            print("🔍 Previewing first 20 valid structures with R1←LEFT and R3→RIGHT orientation...")

            # First pass: collect all molecules
            mols_list = []
            atom_colors_list = []
            rgroup_info_list = []
            smiles_list = []

            for idx, row in df.iterrows():
                if len(mols_list) >= 20:
                    break

                smiles = str(row['Smiles']) if pd.notna(row['Smiles']) else None
                if not smiles:
                    continue

                mol = Chem.MolFromSmiles(smiles)
                if not mol:
                    continue

                atom_colors, rgroup_info = process_rgroups(row, mol)

                mols_list.append(mol)
                atom_colors_list.append(atom_colors)
                rgroup_info_list.append(rgroup_info)
                smiles_list.append(smiles)

            if not mols_list:
                print("❌ No valid structures found in the file")
                return

            # Apply custom orientation to all molecules
            print("🔄 Orienting molecules: R1 (red) ← LEFT, R3 (light blue) → RIGHT...")
            oriented_count = 0
            for i, (mol, atom_colors) in enumerate(zip(mols_list, atom_colors_list)):
                if atom_colors:  # Only orient if we have colored atoms
                    success = align_to_desired_orientation(mol, atom_colors)
                    if success:
                        oriented_count += 1
                        print(f"   ✓ Oriented molecule {i+1}")

            print(f"✅ Successfully oriented {oriented_count}/{len(mols_list)} molecules")

            # Display all oriented molecules
            for idx, (mol, atom_colors, rgroup_info, smiles) in enumerate(zip(mols_list, atom_colors_list, rgroup_info_list, smiles_list)):
                display(HTML(f"<h4>Compound {idx+1}: {smiles}</h4>"))
                display(HTML(f"<div><strong>R-group matches:</strong> {', '.join(rgroup_info)}</div>"))

                # Show legend
                if atom_colors:
                    legend = create_legend(atom_colors)
                    display(HTML(legend))

                # Generate and display colored structure
                img_html = preview_structure(mol, atom_colors)
                display(HTML(img_html))
                display(HTML("<hr>"))

            print(f"🎉 Displayed {len(mols_list)} compounds with consistent R-group orientation")

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            import traceback
            print(traceback.format_exc())

def on_process_click(b):
    with output:
        clear_output()
        if not upload_btn.value:
            print("⚠️ Please upload an Excel file first")
            return

        try:
            file_info = next(iter(upload_btn.value.values()))
            df = pd.read_excel(BytesIO(file_info['content']))
            output_dir = output_dir_input.value
            os.makedirs(output_dir, exist_ok=True)

            print("⚙️ Generating oriented preview images (R1←LEFT, R3→RIGHT)...")
            success_count = 0

            for idx, row in df.iterrows():
                if success_count >= 20:  # Limit to first 20
                    break

                smiles = str(row['Smiles']) if pd.notna(row['Smiles']) else None
                if not smiles:
                    print(f"⚠️ Row {idx+1}: Empty SMILES - skipping")
                    continue
                mol = Chem.MolFromSmiles(smiles)
                if not mol:
                    print(f"⚠️ Row {idx+1}: Invalid SMILES - skipping")
                    continue

                atom_colors, rgroup_info = process_rgroups(row, mol)

                # Apply custom orientation
                if atom_colors:
                    align_to_desired_orientation(mol, atom_colors)

                # Generate image
                from rdkit.Chem.Draw import rdMolDraw2D
                drawer = rdMolDraw2D.MolDraw2DCairo(800, 800)
                draw_options = drawer.drawOptions()
                draw_options.fillHighlights = False
                draw_options.highlightRadius = 0

                highlight_atoms = list(atom_colors.keys())
                highlight_colors = atom_colors.copy()

                # Color bonds between same R-group atoms
                highlight_bonds = []
                bond_colors = {}
                for bond in mol.GetBonds():
                    begin_idx = bond.GetBeginAtomIdx()
                    end_idx = bond.GetEndAtomIdx()
                    if (begin_idx in atom_colors and end_idx in atom_colors and
                        atom_colors[begin_idx] == atom_colors[end_idx]):
                        highlight_bonds.append(bond.GetIdx())
                        bond_colors[bond.GetIdx()] = atom_colors[begin_idx]

                drawer.DrawMolecule(mol,
                                   highlightAtoms=highlight_atoms,
                                   highlightAtomColors=highlight_colors,
                                   highlightBonds=highlight_bonds,
                                   highlightBondColors=bond_colors)

                drawer.FinishDrawing()

                # Save image
                filename = os.path.join(output_dir, f"compound_{idx+1}.png")
                with open(filename, 'wb') as f:
                    f.write(drawer.GetDrawingText())

                success_count += 1
                if success_count <= 5:  # Show first 5 for feedback
                    print(f"✅ Row {idx+1}: Generated oriented image - {', '.join(rgroup_info)}")

            print(f"\n🎉 Successfully processed {success_count} oriented molecules")
            print(f"📁 Images saved to: {os.path.abspath(output_dir)}")

        except Exception as e:
            print(f"❌ Error: {str(e)}")

# Attach event handlers
preview_btn.on_click(on_preview_click)
process_btn.on_click(on_process_click)

In [ ]:
#@title Color with slightly reduced intensed red and green.
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdFMCS, Draw
import os
from io import BytesIO
import base64

# ========== UI SETUP ==========
upload_btn = widgets.FileUpload(description="Upload Excel", accept='.xlsx,.xls', multiple=False)
output_dir_input = widgets.Text(value='preview_output', description='Output folder:')
process_btn = widgets.Button(description="Generate Preview Images", button_style='success')
preview_btn = widgets.Button(description="Preview Structures", button_style='info')
output = widgets.Output()

display(widgets.VBox([
    widgets.HTML("<h2>SMILES to Colored Structure Preview</h2>"),
    widgets.HTML("Upload Excel with columns: <b>Smiles, R1_Reagent, R2_Reagent, R3_Reagent, R4_Reagent</b>"),
    upload_btn,
    output_dir_input,
    widgets.HBox([preview_btn, process_btn]),
    output
]))

# ========== PROCESSING FUNCTIONS ==========
def process_rgroups(row, mol):
    """Identify R-group mappings with proper visualization"""
    atom_colors = {}
    rgroup_info = []

    # Define R-groups with their colors - using lighter colors for better visibility
    rgroups = [
        ('R1', 'R1_Reagent', (1.0, 0.4, 0.4)),    # Lighter Red (was 1.0, 0.0, 0.0)
        ('R2', 'R2_Reagent', (0.4, 0.8, 0.4)),    # Lighter Green (was 0.0, 1.0, 0.0)
        ('R3', 'R3_Reagent', (0.4, 0.6, 1.0)),    # Lighter Blue (was 0.0, 0.0, 1.0)
        ('R4', 'R4_Reagent', (1.0, 0.0, 1.0))     # Magenta
    ]

    for rname, rcol, color in rgroups:
        if rcol in row and pd.notna(row[rcol]):
            reagent_smiles = str(row[rcol])
            reagent_mol = Chem.MolFromSmiles(reagent_smiles)
            if reagent_mol:
                mcs = rdFMCS.FindMCS([mol, reagent_mol])
                if mcs.numAtoms > 0:
                    substruct = Chem.MolFromSmarts(mcs.smartsString)
                    matches = mol.GetSubstructMatches(substruct)
                    for match in matches:
                        for atom_idx in match:
                            if atom_idx not in atom_colors:
                                atom_colors[atom_idx] = color
                    rgroup_info.append(f"{rname}: {len(matches)} matches")
                else:
                    rgroup_info.append(f"{rname}: No common substructure")
            else:
                rgroup_info.append(f"{rname}: Invalid reagent SMILES")
        else:
            rgroup_info.append(f"{rname}: Not provided")

    return atom_colors, rgroup_info

def orient_molecule(mol, atom_colors):
    """Orient molecule so R1 (red) is on left and R3 (blue) is on right"""
    # First generate 2D coordinates
    AllChem.Compute2DCoords(mol)

    # Get atom positions
    conf = mol.GetConformer()

    # Find centroids of R1 (red) and R3 (blue) atoms
    red_atoms = [idx for idx, color in atom_colors.items() if color == (1.0, 0.4, 0.4)]  # Updated red color
    blue_atoms = [idx for idx, color in atom_colors.items() if color == (0.4, 0.6, 1.0)]  # Updated blue color

    if red_atoms and blue_atoms:
        # Calculate centroids
        red_centroid_x = sum(conf.GetAtomPosition(idx).x for idx in red_atoms) / len(red_atoms)
        blue_centroid_x = sum(conf.GetAtomPosition(idx).x for idx in blue_atoms) / len(blue_atoms)

        # If R1 (red) is not on the left side, rotate the molecule
        if red_centroid_x > blue_centroid_x:
            # Rotate 180 degrees around the center
            center_x = sum(conf.GetAtomPosition(i).x for i in range(mol.GetNumAtoms())) / mol.GetNumAtoms()
            center_y = sum(conf.GetAtomPosition(i).y for i in range(mol.GetNumAtoms())) / mol.GetNumAtoms()

            for i in range(mol.GetNumAtoms()):
                pos = conf.GetAtomPosition(i)
                new_x = 2 * center_x - pos.x  # Mirror across center
                new_y = pos.y
                conf.SetAtomPosition(i, (new_x, new_y, 0))

    return mol

def create_custom_template():
    """Create a simple template with R1 on left and R3 on right"""
    # Create a simple scaffold that can serve as a template
    # This is a generic scaffold that encourages left-right orientation
    template_smiles = "C(C)(C)C"  # Simple branched molecule that encourages left-right orientation
    template_mol = Chem.MolFromSmiles(template_smiles)
    if template_mol:
        AllChem.Compute2DCoords(template_mol)
    return template_mol

def align_to_desired_orientation(mol, atom_colors):
    """Align molecule to have R1 on left and R3 on right"""
    try:
        # First try to use a custom template
        template_mol = create_custom_template()
        if template_mol:
            AllChem.GenerateDepictionMatching2DStructure(mol, template_mol)

        # Then apply our custom orientation
        mol = orient_molecule(mol, atom_colors)
        return True
    except:
        # Fallback: just use custom orientation
        try:
            mol = orient_molecule(mol, atom_colors)
            return True
        except:
            AllChem.Compute2DCoords(mol)
            return False

def preview_structure(mol, atom_colors):
    """Generate preview with custom R-group coloring (no shadows)"""
    from rdkit.Chem.Draw import rdMolDraw2D

    # Create drawer
    drawer = rdMolDraw2D.MolDraw2DCairo(400, 400)
    draw_options = drawer.drawOptions()

    # CRITICAL: Disable all highlight effects to remove shadows
    draw_options.fillHighlights = False
    draw_options.highlightRadius = 0

    # Prepare highlight information
    highlight_atoms = list(atom_colors.keys())
    highlight_colors = {}
    for atom_idx, color in atom_colors.items():
        highlight_colors[atom_idx] = color

    # Also color bonds between atoms of the same R-group
    highlight_bonds = []
    bond_colors = {}
    for bond in mol.GetBonds():
        begin_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        if (begin_idx in atom_colors and end_idx in atom_colors and
            atom_colors[begin_idx] == atom_colors[end_idx]):
            highlight_bonds.append(bond.GetIdx())
            bond_colors[bond.GetIdx()] = atom_colors[begin_idx]

    # Draw the molecule
    drawer.DrawMolecule(mol,
                       highlightAtoms=highlight_atoms,
                       highlightAtomColors=highlight_colors,
                       highlightBonds=highlight_bonds,
                       highlightBondColors=bond_colors)

    drawer.FinishDrawing()

    # Convert to base64 for display
    img_data = drawer.GetDrawingText()
    img_str = base64.b64encode(img_data).decode()
    return f'<img src="data:image/png;base64,{img_str}" />'

def create_legend(atom_colors):
    """Create a proper legend showing which colors correspond to which R-groups"""
    color_mapping = {
        (1.0, 0.4, 0.4): "R1 (Light Red) ← LEFT",     # Updated description
        (0.4, 0.8, 0.4): "R2 (Light Green)",          # Updated description
        (0.4, 0.6, 1.0): "R3 (Light Blue) → RIGHT",   # Updated description
        (1.0, 0.0, 1.0): "R4 (Magenta)"
    }

    used_colors = set(atom_colors.values())
    legend_html = "<div style='margin: 10px 0; padding: 10px; border: 1px solid #ccc; background: #f9f9f9;'>"
    legend_html += "<strong>R-Group Colors (Oriented: R1←LEFT, R3→RIGHT):</strong><br>"

    for color in used_colors:
        color_name = color_mapping.get(tuple(color), "Unknown R-group")
        hex_color = "#{:02x}{:02x}{:02x}".format(
            int(color[0] * 255),
            int(color[1] * 255),
            int(color[2] * 255)
        )
        legend_html += f"<span style='color: {hex_color}; font-weight: bold; margin-right: 15px;'>■ {color_name}</span>"

    legend_html += "</div>"
    return legend_html

# ========== EVENT HANDLERS ==========
def on_preview_click(b):
    with output:
        clear_output()
        if not upload_btn.value:
            print("⚠️ Please upload an Excel file first")
            return

        try:
            file_info = next(iter(upload_btn.value.values()))
            df = pd.read_excel(BytesIO(file_info['content']))
            print("🔍 Previewing first 20 valid structures with R1←LEFT and R3→RIGHT orientation...")

            # First pass: collect all molecules
            mols_list = []
            atom_colors_list = []
            rgroup_info_list = []
            smiles_list = []

            for idx, row in df.iterrows():
                if len(mols_list) >= 20:
                    break

                smiles = str(row['Smiles']) if pd.notna(row['Smiles']) else None
                if not smiles:
                    continue

                mol = Chem.MolFromSmiles(smiles)
                if not mol:
                    continue

                atom_colors, rgroup_info = process_rgroups(row, mol)

                mols_list.append(mol)
                atom_colors_list.append(atom_colors)
                rgroup_info_list.append(rgroup_info)
                smiles_list.append(smiles)

            if not mols_list:
                print("❌ No valid structures found in the file")
                return

            # Apply custom orientation to all molecules
            print("🔄 Orienting molecules: R1 (light red) ← LEFT, R3 (light blue) → RIGHT...")
            oriented_count = 0
            for i, (mol, atom_colors) in enumerate(zip(mols_list, atom_colors_list)):
                if atom_colors:  # Only orient if we have colored atoms
                    success = align_to_desired_orientation(mol, atom_colors)
                    if success:
                        oriented_count += 1
                        print(f"   ✓ Oriented molecule {i+1}")

            print(f"✅ Successfully oriented {oriented_count}/{len(mols_list)} molecules")

            # Display all oriented molecules
            for idx, (mol, atom_colors, rgroup_info, smiles) in enumerate(zip(mols_list, atom_colors_list, rgroup_info_list, smiles_list)):
                display(HTML(f"<h4>Compound {idx+1}: {smiles}</h4>"))
                display(HTML(f"<div><strong>R-group matches:</strong> {', '.join(rgroup_info)}</div>"))

                # Show legend
                if atom_colors:
                    legend = create_legend(atom_colors)
                    display(HTML(legend))

                # Generate and display colored structure
                img_html = preview_structure(mol, atom_colors)
                display(HTML(img_html))
                display(HTML("<hr>"))

            print(f"🎉 Displayed {len(mols_list)} compounds with consistent R-group orientation")

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            import traceback
            print(traceback.format_exc())

def on_process_click(b):
    with output:
        clear_output()
        if not upload_btn.value:
            print("⚠️ Please upload an Excel file first")
            return

        try:
            file_info = next(iter(upload_btn.value.values()))
            df = pd.read_excel(BytesIO(file_info['content']))
            output_dir = output_dir_input.value
            os.makedirs(output_dir, exist_ok=True)

            print("⚙️ Generating oriented preview images (R1←LEFT, R3→RIGHT)...")
            success_count = 0

            for idx, row in df.iterrows():
                if success_count >= 20:  # Limit to first 20
                    break

                smiles = str(row['Smiles']) if pd.notna(row['Smiles']) else None
                if not smiles:
                    print(f"⚠️ Row {idx+1}: Empty SMILES - skipping")
                    continue
                mol = Chem.MolFromSmiles(smiles)
                if not mol:
                    print(f"⚠️ Row {idx+1}: Invalid SMILES - skipping")
                    continue

                atom_colors, rgroup_info = process_rgroups(row, mol)

                # Apply custom orientation
                if atom_colors:
                    align_to_desired_orientation(mol, atom_colors)

                # Generate image
                from rdkit.Chem.Draw import rdMolDraw2D
                drawer = rdMolDraw2D.MolDraw2DCairo(800, 800)
                draw_options = drawer.drawOptions()
                draw_options.fillHighlights = False
                draw_options.highlightRadius = 0

                highlight_atoms = list(atom_colors.keys())
                highlight_colors = atom_colors.copy()

                # Color bonds between same R-group atoms
                highlight_bonds = []
                bond_colors = {}
                for bond in mol.GetBonds():
                    begin_idx = bond.GetBeginAtomIdx()
                    end_idx = bond.GetEndAtomIdx()
                    if (begin_idx in atom_colors and end_idx in atom_colors and
                        atom_colors[begin_idx] == atom_colors[end_idx]):
                        highlight_bonds.append(bond.GetIdx())
                        bond_colors[bond.GetIdx()] = atom_colors[begin_idx]

                drawer.DrawMolecule(mol,
                                   highlightAtoms=highlight_atoms,
                                   highlightAtomColors=highlight_colors,
                                   highlightBonds=highlight_bonds,
                                   highlightBondColors=bond_colors)

                drawer.FinishDrawing()

                # Save image
                filename = os.path.join(output_dir, f"compound_{idx+1}.png")
                with open(filename, 'wb') as f:
                    f.write(drawer.GetDrawingText())

                success_count += 1
                if success_count <= 5:  # Show first 5 for feedback
                    print(f"✅ Row {idx+1}: Generated oriented image - {', '.join(rgroup_info)}")

            print(f"\n🎉 Successfully processed {success_count} oriented molecules")
            print(f"📁 Images saved to: {os.path.abspath(output_dir)}")

        except Exception as e:
            print(f"❌ Error: {str(e)}")

# Attach event handlers
preview_btn.on_click(on_preview_click)
process_btn.on_click(on_process_click)